In [3]:
import numpy as np
from qiskit import QuantumCircuit, ClassicalRegister, QuantumRegister
from qiskit import Aer, execute, quantum_info

In [9]:
# Unitary 4 Qubit gate with Rz's full entanglement
def even_gate(theta, index):
    name = "Ue"+str(index)+"("+str(theta)+")"
    u_even = QuantumCircuit(4, name=name)
    u_even.rz(theta,0)
    u_even.rz(theta,1)
    u_even.rz(theta,2)
    u_even.rz(theta,3)

    u_even.cz(0,1)
    u_even.cz(0,2)
    u_even.cz(0,3)
    u_even.cz(1,2)
    u_even.cz(0,3)
    u_even.cz(2,3)

    return u_even

# Unitary 4 Qubit gate with Rx's and no entanglement
def odd_gate(theta, index):
    name = "Uo"+str(index)+"("+str(theta)+")"
    u_odd = QuantumCircuit(4, name=name)
    u_odd.rx(theta,0)
    u_odd.rx(theta,1)
    u_odd.rx(theta,2)
    u_odd.rx(theta,3)

    return u_odd

In [10]:
L = 3

phi = quantum_info.random_statevector(16)

circuit = QuantumCircuit(4)

for i in range(L):
    theta = np.random.uniform(0, 2*np.pi)
    circuit.append(even_gate(theta, i), [0, 1, 2, 3])
    circuit.append(odd_gate(theta, i), [0, 1, 2, 3])
    
circuit.draw()

┌──────────────────────────┐┌──────────────────────────┐»
q_0: ┤0                         ├┤0                         ├»
     │                          ││                          │»
q_1: ┤1                         ├┤1                         ├»
     │  Ue0(0.8737271621362179) ││  Uo0(0.8737271621362179) │»
q_2: ┤2                         ├┤2                         ├»
     │                          ││                          │»
q_3: ┤3                         ├┤3                         ├»
     └──────────────────────────┘└──────────────────────────┘»
«     ┌──────────────────────────┐┌──────────────────────────┐»
«q_0: ┤0                         ├┤0                         ├»
«     │                          ││                          │»
«q_1: ┤1                         ├┤1                         ├»
«     │  Ue1(0.3634669084648078) ││  Uo1(0.3634669084648078) │»
«q_2: ┤2                         ├┤2                         ├»
«     │                          ││                          │»
«q_3: ┤3                         ├┤3                         ├»
«     └──────────────────────────┘└──────────────────────────┘»
«     ┌─────────────────────────┐┌─────────────────────────┐
«q_0: ┤0                        ├┤0                        ├
«     │                         ││                         │
«q_1: ┤1                        ├┤1                        ├
«     │  Ue2(5.287601560843217) ││  Uo2(5.287601560843217) │
«q_2: ┤2                        ├┤2                        ├
«     │                         ││                         │
«q_3: ┤3                        ├┤3                        ├
«     └─────────────────────────┘└─────────────────────────┘

In [29]:
simulator = Aer.get_backend("statevector_simulator")

job = execute(circuit, backend=simulator, shots=1024)
result = job.result()
phi_theta = result.get_statevector(circuit)
print(phi_theta)

[ 0.67916225-0.5060647j  -0.07766422+0.1016488j  -0.07766422+0.1016488j
  0.06417263-0.16355447j  0.00267306+0.05143571j  0.11266673-0.09793946j
  0.11266673-0.09793946j  0.2247239 +0.05571224j -0.18110621+0.13247225j
  0.05445673+0.02481664j  0.05445673+0.02481664j -0.06745192+0.10243677j
  0.07522207-0.15772319j  0.04838722+0.04549086j  0.04838722+0.04549086j
 -0.0299657 +0.07350253j]


In [33]:
np.linalg.norm(phi_theta - phi.data)

1.3229642780340438